In [3]:
!mkdir tensorflow-demo
!cd tensorflow-demo

In [4]:
!python -m venv tensorflow-demo

In [5]:
"tensorflow-demo/Scripts/activate.bat"

'tensorflow-demo/Scripts/activate.bat'

In [9]:
!pip install keras
!pip install numpy
!pip install Pillow
!pip install scikit-learn
!pip install scipy
!pip install sklearn
!pip install tensorflow

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/332.0 MB 7.2 MB/s eta 0:00:47
    --------------------------------------- 5.8/332.0 MB 19.2 MB/s eta 0:00:17
   - -------------------------------------- 12.1/332.0 MB 24.1 MB/s eta 0:00:14
   -- ------------------------------------- 18.4/332.0 MB 26.0 MB/s eta 0:00:13
   -- ------------------------------------- 22.0/332.0 MB 24.1 MB/s eta 0:00:13
   --- ------------------------------------ 29.6/332.0 MB 26.6 MB/s eta 0:00:12
   ---- ----------------------------------- 36.2/332.0 MB 27.4 MB/s eta 0:00:11
   ----- ---------------------------------- 42.2/332.0 MB 27.7 MB/s eta 0:00:11
   ----- ---------------------------------- 49.0/332.0 MB 28.3 MB/s eta 0:00:11
   ------ --------------------------------- 55.6/332.0 MB 28.5 MB/s eta 0:00:10
   ------- -------------------------------- 61.6/332.0 MB 28.7 MB/s eta 0:00:10
   -------- ------------------------------- 68.2/332

In [11]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from PIL import Image
tf.compat.v1.disable_v2_behavior() 

In [13]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print (len(x_train))
print (len(x_test))
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = [[i] for i in y_train]
y_test = [[i] for i in y_test]
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)
enc.fit(y_train)
y_train =  enc.transform(y_train)
y_test = enc.transform(y_test)

60000
10000


In [14]:
n_input = 784 # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10 # output layer (0-9 digits)

In [15]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [16]:
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_output])
keep_prob = tf.compat.v1.placeholder(tf.float32)

In [17]:
weights = {
    'w1': tf.Variable(tf.random.truncated_normal([n_input,
    n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.random.truncated_normal([n_hidden1,
    n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.random.truncated_normal([n_hidden2,
    n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.random.truncated_normal([n_hidden3,
    n_output], stddev=0.1)),
}

In [18]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
 }

In [23]:
# الطبقة الأولى
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])

# الطبقة الثانية
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])

# الطبقة الثالثة
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])

# طبقة الإسقاط (dropout)
layer_drop = tf.nn.dropout(layer_3, keep_prob)

# طبقة الإخراج
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

In [24]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.compat.v1.train.AdamOptimizer(1e4).minimize(cross_entropy)

In [25]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [26]:
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)

In [29]:
#    ةريغصتاعفد ىلعبيردتلا
for i in range(n_iterations):
    startbatch = (i*batch_size) % len(x_train)
    endbatch = ((i+1)*batch_size) % len(x_train)
    batch_x = np.array(x_train[startbatch:endbatch])
    batch_x = batch_x.reshape(batch_size, -1)
    batch_y = y_train[startbatch:endbatch]  # <-- تم حذف .toarray()
    if batch_x.shape != (128, 784):
        continue
    sess.run(train_step, feed_dict={
        X: batch_x, Y: (batch_y), keep_prob: dropout})
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy))


Iteration 0 	| Loss = 2.686494e+22 	| Accuracy = 0.0703125
Iteration 100 	| Loss = 1.1505801e+22 	| Accuracy = 0.7734375
Iteration 200 	| Loss = 3.2802452e+21 	| Accuracy = 0.8203125
Iteration 300 	| Loss = 1.10993964e+21 	| Accuracy = 0.90625
Iteration 400 	| Loss = 1.9657095e+21 	| Accuracy = 0.75
Iteration 500 	| Loss = 5.3082296e+20 	| Accuracy = 0.875
Iteration 600 	| Loss = 9.913923e+20 	| Accuracy = 0.828125
Iteration 700 	| Loss = 3.2085037e+20 	| Accuracy = 0.875
Iteration 800 	| Loss = 9.773555e+20 	| Accuracy = 0.8046875
Iteration 900 	| Loss = 4.5311096e+20 	| Accuracy = 0.828125


In [31]:
x_test = x_test.reshape(-1,784)
test_accuracy = sess.run(accuracy, feed_dict={X: x_test, Y: y_test, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.8501


In [50]:
!(tensorflow-demo) $ python main.py

$ was unexpected at this time.


In [ ]:
img = np.array(Image.open(r"D:\MyDesktop\mnist_images\13_label6.png").convert('L')).ravel()

In [95]:
prediction = sess.run(tf.argmax(output_layer, 1), feed_dict={X: 
[img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 6
